In [1]:
import classifiers
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import pickle

## Load Models

### TF-IDF

In [2]:
with open ("tfidf.pk", 'rb') as fin:
    tfidf = pickle.load(fin)

In [3]:
tfidf

TfidfVectorizer(max_df=5, ngram_range=(1, 3), stop_words='english')

### Word2Vec CBOW

In [4]:
cbow = Word2Vec.load("cbow.model")

In [5]:
cbow

### Word2Vec SkipGram

In [6]:
skg = Word2Vec.load("skg.model")

In [7]:
skg

## Importing Data

In [8]:
df = pd.read_csv("processed_train.csv")

In [9]:
df

,Unnamed: 0,id,tweet,subtask_a,subtask_b,subtask_c
0,0,86426,"['ask', 'native', 'american', 'take']",OFF,UNT,NaN
1,1,90194,"['go', 'home', '’', 'drunk', 'maga', 'trump', ...",OFF,TIN,IND
2,2,16820,"['amazon', 'investigating', 'chinese', 'employ...",NOT,NaN,NaN
3,3,62688,"['someone', 'shouldve', 'taken', 'piece', 'shi...",OFF,UNT,NaN
4,4,43605,"['obama', 'wanted', 'liberal', 'amp', 'illegal...",NOT,NaN,NaN
...,...,...,...,...,...,...
13235,13235,95338,"['sometimes', 'get', 'strong', 'vibe', 'people...",OFF,TIN,IND
13236,13236,67210,"['benidorm', '✅', 'creamfields', '✅', 'maga', ...",NOT,NaN,NaN
13237,13237,82921,"['report', 'garbage', 'dont', 'give', 'crap']",OFF,TIN,OTH
13238,13238,27429,['pussy'],OFF,UNT,NaN


### Removing Redundant Axes

In [10]:
df = df.drop(['Unnamed: 0', 'subtask_b', 'subtask_c', 'id'], axis=1)

In [11]:
df

,tweet,subtask_a
0,"['ask', 'native', 'american', 'take']",OFF
1,"['go', 'home', '’', 'drunk', 'maga', 'trump', ...",OFF
2,"['amazon', 'investigating', 'chinese', 'employ...",NOT
3,"['someone', 'shouldve', 'taken', 'piece', 'shi...",OFF
4,"['obama', 'wanted', 'liberal', 'amp', 'illegal...",NOT
...,...,...
13235,"['sometimes', 'get', 'strong', 'vibe', 'people...",OFF
13236,"['benidorm', '✅', 'creamfields', '✅', 'maga', ...",NOT
13237,"['report', 'garbage', 'dont', 'give', 'crap']",OFF
13238,['pussy'],OFF


### Renaming Columns

In [12]:
df = df.rename(columns={'subtask_a': 'Offensive'})

In [13]:
df

,tweet,Offensive
0,"['ask', 'native', 'american', 'take']",OFF
1,"['go', 'home', '’', 'drunk', 'maga', 'trump', ...",OFF
2,"['amazon', 'investigating', 'chinese', 'employ...",NOT
3,"['someone', 'shouldve', 'taken', 'piece', 'shi...",OFF
4,"['obama', 'wanted', 'liberal', 'amp', 'illegal...",NOT
...,...,...
13235,"['sometimes', 'get', 'strong', 'vibe', 'people...",OFF
13236,"['benidorm', '✅', 'creamfields', '✅', 'maga', ...",NOT
13237,"['report', 'garbage', 'dont', 'give', 'crap']",OFF
13238,['pussy'],OFF


### Converting Offensive to Numerical Value

In [14]:
def off(cls):
    if cls =='OFF':
        return 1
    return 0

In [15]:
df["Offensive"] = df["Offensive"].apply(off)

In [16]:
df

,tweet,Offensive
0,"['ask', 'native', 'american', 'take']",1
1,"['go', 'home', '’', 'drunk', 'maga', 'trump', ...",1
2,"['amazon', 'investigating', 'chinese', 'employ...",0
3,"['someone', 'shouldve', 'taken', 'piece', 'shi...",1
4,"['obama', 'wanted', 'liberal', 'amp', 'illegal...",0
...,...,...
13235,"['sometimes', 'get', 'strong', 'vibe', 'people...",1
13236,"['benidorm', '✅', 'creamfields', '✅', 'maga', ...",0
13237,"['report', 'garbage', 'dont', 'give', 'crap']",1
13238,['pussy'],1


In [17]:
df['tweet']

0                    ['ask', 'native', 'american', 'take']
1        ['go', 'home', '’', 'drunk', 'maga', 'trump', ...
2        ['amazon', 'investigating', 'chinese', 'employ...
3        ['someone', 'shouldve', 'taken', 'piece', 'shi...
4        ['obama', 'wanted', 'liberal', 'amp', 'illegal...
                               ...                        
13235    ['sometimes', 'get', 'strong', 'vibe', 'people...
13236    ['benidorm', '✅', 'creamfields', '✅', 'maga', ...
13237        ['report', 'garbage', 'dont', 'give', 'crap']
13238                                            ['pussy']
13239    ['spanishrevenge', 'v', 'justice', 'human', 'r...
Name: tweet, Length: 13240, dtype: object

## Fitting Word Models on Data

In [18]:
X_train, X_test, y_train, y_test = train_test_split(df['tweet'], df['Offensive'], stratify=df['Offensive'], random_state=0)

### TF-IDF

In [19]:
X_train_tf = tfidf.transform(X_train)

In [20]:
X_test_tf = tfidf.transform(X_test)

### Sentence Vectoriser

Finding the average vector for a given document

In [27]:
def sent_vect(sent, model):
    vec = np.zeros(400)
    num = 0
    for w in sent:
        temp_vec = model.wv.get_vector(w)
        #print(temp_vec)
        vec = np.add(vec, temp_vec)
        num += 1
        
    return vec / np.sqrt(vec.dot(vec))

### CBOW

In [28]:
X_train_cbow = X_train.apply(sent_vect, model=cbow)

KeyError: "word '[' not in vocabulary"

In [23]:
X_train_cbow

3759     [nan, nan, nan, nan, nan, nan, nan, nan, nan, ...
1623     [nan, nan, nan, nan, nan, nan, nan, nan, nan, ...
8650     [nan, nan, nan, nan, nan, nan, nan, nan, nan, ...
7897     [nan, nan, nan, nan, nan, nan, nan, nan, nan, ...
365      [nan, nan, nan, nan, nan, nan, nan, nan, nan, ...
                               ...                        
11365    [nan, nan, nan, nan, nan, nan, nan, nan, nan, ...
8409     [nan, nan, nan, nan, nan, nan, nan, nan, nan, ...
7883     [nan, nan, nan, nan, nan, nan, nan, nan, nan, ...
10443    [nan, nan, nan, nan, nan, nan, nan, nan, nan, ...
10972    [nan, nan, nan, nan, nan, nan, nan, nan, nan, ...
Name: tweet, Length: 9930, dtype: object

In [24]:
X_train

3759             ['friend', 'father', 'safe', 'talk', '”']
1623                                   ['president', 'im']
8650     ['depending', 'government', 'politician', 'fur...
7897     ['im', 'sure', 'really', 'low', 'carb', 'diet'...
365                   ['welcome', 'why', 'james', 'cryin']
                               ...                        
11365    ['nra', 'supported', 'gun', 'control', 'reagan...
8409     ['quite', 'good', 'faking', 'must', 'done', 'n...
7883                                             ['thank']
10443    ['abundantly', 'clear', 'lack', 'common', 'sen...
10972                 ['there', 'brexit', '👇', '🏻', 'url']
Name: tweet, Length: 9930, dtype: object

In [25]:
type(df["tweet"][0])

str

In [26]:
sent_vect(df["tweet"][0], cbow)

<ipython-input-21-ef0a14eb220b>:13: RuntimeWarning: invalid value encountered in true_divide
  return vec / np.sqrt(vec.dot(vec))


array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, na